In [8]:
import requests
import nltk
import re
from nltk.corpus import stopwords
import numpy as np
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
# Download 'punkt_tab' data package
nltk.download('punkt_tab') # Download the required data package. This data is not installed by default


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
# Load the dataset
url = "https://huggingface.co/datasets/sleeping-ai/TEKGEN-Wiki"
response = requests.get(url)
data = response.text
# Save the data to a file
with open("dataset.txt", "w", encoding="utf-8") as file:
    file.write(data)

In [10]:
## Load the dataset from file
with open('dataset.txt', 'r', encoding='utf-8') as file:
    text = file.read()
# Tokenization
tokens = nltk.word_tokenize(text)

In [11]:
# Lowercasing
tokens = [token.lower() for token in tokens]

In [12]:
# Removing punctuation and special characters
tokens = [re.sub(r'\W+', '', token) for token in tokens if re.sub(r'\W+', '', token)]

# Removing stop words (optional)
stop_words = set(stopwords.words('english'))
tokens = [token for token in tokens if token not in stop_words]

# Add a special token for unknown words
tokens.append('')


In [13]:
# Numericalization
vocab = list(set(tokens))
word2index = {word: i for i, word in enumerate(vocab)}
index2word = {i: word for i, word in enumerate(vocab)}

In [14]:
# Creating sequences
sequence_length = 5
sequences = []
for i in range(len(tokens) - sequence_length):
    sequences.append(tokens[i:i + sequence_length])


In [15]:
# Convert sequences to numerical indices
input_sequences = []
for sequence in sequences:
    input_sequences.append([word2index[word] for word in sequence])


In [16]:
# Convert to numpy array
input_sequences = np.array(input_sequences)

print(f"Total sequences: {len(input_sequences)}")

Total sequences: 21210


In [17]:
import torch
import torch.nn as nn

class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, prev_state):
        x = self.embedding(x)
        x, state = self.lstm(x, prev_state)
        x = self.fc(x)
        return x, state

    def init_state(self, batch_size=1):
        return (torch.zeros(2, batch_size, self.lstm.hidden_size),
                torch.zeros(2, batch_size, self.lstm.hidden_size))

In [18]:
import torch.optim as optim

# Hyperparameters
embedding_dim = 50
hidden_dim = 100
vocab_size = len(vocab)
batch_size = 32
epochs = 20

# Model, loss function, optimizer
model = LanguageModel(vocab_size, embedding_dim, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, input_sequences, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for i in range(0, len(input_sequences) - batch_size, batch_size):
            inputs = torch.tensor(input_sequences[i:i + batch_size, :-1], dtype=torch.long)
            targets = torch.tensor(input_sequences[i:i + batch_size, 1:], dtype=torch.long)

            optimizer.zero_grad()
            state_h, state_c = model.init_state(batch_size)
            outputs, _ = model(inputs, (state_h, state_c))
            loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / (len(input_sequences) // batch_size)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

# Train the model
train_model(model, input_sequences, criterion, optimizer, epochs)

Epoch 1/20, Loss: 6.6853
Epoch 2/20, Loss: 5.1809
Epoch 3/20, Loss: 4.5664
Epoch 4/20, Loss: 4.0004
Epoch 5/20, Loss: 3.4469
Epoch 6/20, Loss: 2.9702
Epoch 7/20, Loss: 2.5734
Epoch 8/20, Loss: 2.2383
Epoch 9/20, Loss: 1.9564
Epoch 10/20, Loss: 1.7181
Epoch 11/20, Loss: 1.5147
Epoch 12/20, Loss: 1.3463
Epoch 13/20, Loss: 1.2080
Epoch 14/20, Loss: 1.0970
Epoch 15/20, Loss: 1.0062
Epoch 16/20, Loss: 0.9317
Epoch 17/20, Loss: 0.8736
Epoch 18/20, Loss: 0.8266
Epoch 19/20, Loss: 0.7885
Epoch 20/20, Loss: 0.7582


In [19]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'model.pth')
print("Model saved successfully.")

Model saved successfully.


In [20]:
def generate_text(model, start_text, max_length=50):
    model.eval()
    words = start_text.split()
    state_h, state_c = model.init_state(batch_size=1)

    for _ in range(max_length):
        # Use the word2index dictionary with a fallback to  if the word is not found
        x = torch.tensor([[word2index.get(w, word2index['']) for w in words]], dtype=torch.long)
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(index2word[word_index])

    return ' '.join(words)

# Test the model with a sample input
start_text = "harry potter is"
generated_text = generate_text(model, start_text)
print("Generated Text:")
print(generated_text)

Generated Text:
harry potter is roundedfull textsm bordergray100 transitiontransform grouphover textgray400 dark fillgray50080 rx 2 x 528 25 width 112 height 4 fillopacity 1 rect rect class fillindigo500 cursorpointer x rounded path path class originbottomright transitiontransform grouphover rotate6 60539 ranging fear shame hope god love rzayev 0566711 professional expand 30353 2725 68042 331486 dark original


In [23]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.9 MB/s eta 0:00:00


In [24]:
import streamlit as st

# Define the model class (this should match your trained model's class)
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, prev_state):
        x = self.embedding(x)
        x, state = self.lstm(x, prev_state)
        x = self.fc(x)
        return x, state

    def init_state(self, batch_size=1):
        return (torch.zeros(2, batch_size, self.lstm.hidden_size),
                torch.zeros(2, batch_size, self.lstm.hidden_size))
# Load the trained model
model = LanguageModel(vocab_size=len(vocab), embedding_dim=50, hidden_dim=100)
model.load_state_dict(torch.load('model.pth'))
model.eval()

# Define the text generation function
def generate_text(model, start_text, max_length=50):
    model.eval()
    words = start_text.split()
    state_h, state_c = model.init_state(batch_size=1)

    for _ in range(max_length):
        # Use the word2index dictionary with a fallback to  if the word is not found
        x = torch.tensor([[word2index.get(w, word2index['']) for w in words]], dtype=torch.long)
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(index2word[word_index])

    return ' '.join(words)

# Streamlit app interface
st.title("Text Generation with Language Model")
st.write("Enter a text prompt and the model will generate a continuation of the text.")

# Input box for user to type in a text prompt
user_input = st.text_input("Enter text prompt:", "Harry Potter is")

# Generate text based on user input
if user_input:
    with st.spinner('Generating text...'):
        generated_text = generate_text(model, user_input)
        st.success("Generated Text:")
        st.write(generated_text)

<ipython-input-24-9a650539a095>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))
2025-01-26 14:49:46.584 WARNING streamlit.ru